In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

# Establishing a connection to MySQL
try:
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",  # Replace with your MySQL username
        password="ank5097",  # Replace with your MySQL password
        database="notetaking"  # Replace with your MySQL database name
    )
except mysql.connector.Error as e:
    print("MySQL Connection error:", e)
    exit()

# Function to open registration page
def open_registration():
    registration_window = tk.Toplevel(root)
    registration_window.title("Registration")
    registration_window.geometry("400x600")  

    # Labels and Entry fields for registration
    tk.Label(registration_window, text="Username:").pack()
    username_entry = tk.Entry(registration_window)
    username_entry.pack()

    tk.Label(registration_window, text="Email:").pack()
    email_entry = tk.Entry(registration_window)
    email_entry.pack()

    tk.Label(registration_window, text="Phone Number:").pack()
    phone_entry = tk.Entry(registration_window)
    phone_entry.pack()

    tk.Label(registration_window, text="Password:").pack()
    password_entry = tk.Entry(registration_window, show="*")
    password_entry.pack()

    tk.Label(registration_window, text="Confirm Password:").pack()
    confirm_password_entry = tk.Entry(registration_window, show="*")
    confirm_password_entry.pack()

    def register():
        # Retrieve values from entry fields
        username = username_entry.get()
        email = email_entry.get()
        phone = phone_entry.get()
        password = password_entry.get()
        confirm_password = confirm_password_entry.get()

        # Validate password match
        if password != confirm_password:
            messagebox.showerror("Error", "Passwords do not match")
            return

        # Insert into database
        cursor = db_connection.cursor()
        insert_query = "INSERT INTO user (username, email, phone, password) VALUES (%s, %s, %s, %s)"
        user_data = (username, email, phone, password)
        try:
            cursor.execute(insert_query, user_data)
            db_connection.commit()
            messagebox.showinfo("Registration", "Registration successful!")
            registration_window.destroy()  # Close registration window after successful registration
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error registering user: {err}")
        finally:
            cursor.close()

    tk.Button(registration_window, text="Register", command=register).pack()

# Function to open note window
def open_note_window(id):
    note_window = tk.Toplevel(root)
    note_window.title("Note Taking")

    def show_note_content(note_id):
        cursor = db_connection.cursor()
        query = "SELECT title, content FROM t WHERE id = %s"
        cursor.execute(query, (note_id,))
        result = cursor.fetchone()
        if result:
            title, content = result
            content_window = tk.Toplevel(note_window)
            content_window.title(title)
            tk.Label(content_window, text=content).pack()
        cursor.close()

    cursor = db_connection.cursor()
    query = "SELECT id, title FROM t WHERE id = %s"
    cursor.execute(query, (id,))
    notes = cursor.fetchall()

    for note in notes:
        note_id, title = note
        tk.Button(note_window, text=title, command=lambda note_id=note_id: show_note_content(note_id)).pack()

    cursor.close()

    # Labels and Entry fields for note
    tk.Label(note_window, text="Title:").pack()
    title_entry = tk.Entry(note_window)
    title_entry.pack()

    tk.Label(note_window, text="Content:").pack()
    content_text = tk.Text(note_window, height=10, width=50)
    content_text.pack()

    # Function to save note to database
    def save_note():
        # Retrieve values from entry fields
        title = title_entry.get()
        content = content_text.get("1.0", tk.END).strip()

        if not title or not content:
            messagebox.showerror("Error", "Please enter both title and content.")
            return

        # Insert into database
        cursor = db_connection.cursor()
        insert_query = "INSERT INTO t (id, title, content) VALUES (%s, %s, %s)"
        note_data = (id, title, content)
        try:
            cursor.execute(insert_query, note_data)
            db_connection.commit()
            messagebox.showinfo("Success", "Note added successfully!")
            title_entry.delete(0, tk.END)
            content_text.delete("1.0", tk.END)
            # Refresh note list after adding new note
            note_window.destroy()
            open_note_window(id)
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error adding note: {err}")
        finally:
            cursor.close()

    # Button to save note
    tk.Button(note_window, text="Save Note", command=save_note).pack()

# Function to check login credentials
def login():
    # Retrieve values from entry fields
    username = username_entry.get()
    password = password_entry.get()

    # Query database for user
    cursor = db_connection.cursor()
    query = "SELECT id FROM user WHERE username = %s AND password = %s"
    user_data = (username, password)
    cursor.execute(query, user_data)
    result = cursor.fetchone()

    if result:
        messagebox.showinfo("Login", "Login successful!")
        id = result[0]
        open_note_window(id)  # Open note window for logged-in user
    else:
        messagebox.showerror("Login failed", "Invalid username or password")

    cursor.close()

# Main Tkinter window
root = tk.Tk()
root.title("Login")
root.geometry("400x600")

# Labels and Entry fields for login
tk.Label(root, text="Username:").pack()
username_entry = tk.Entry(root)
username_entry.pack()

tk.Label(root, text="Password:").pack()
password_entry = tk.Entry(root, show="*")
password_entry.pack()

tk.Button(root, text="Login", command=login).pack()
tk.Button(root, text="Register", command=open_registration).pack()

root.mainloop()
